In [9]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate, LLMMathChain
from langchain.utilities import GoogleSearchAPIWrapper
from getpass import getpass

In [11]:
openai_api_key = getpass()

In [12]:
template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summry_chain = LLMChain(
    llm=OpenAI(openai_api_key=openai_api_key),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)

In [13]:
llm_math_chain = LLMMathChain.from_llm(llm=OpenAI(temperature=0, verbose=True, openai_api_key=openai_api_key), verbose=True)

In [14]:
tools = [
        Tool(
            name="Calculator",
            func=llm_math_chain.run,
            description="useful for when you need to answer questions about math",
        ),
        Tool(
            name="Summary",
            func=summry_chain.run,
            description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
        ),
]

In [15]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [20]:
llm_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=openai_api_key, verbose=True), prompt=prompt, verbose=True)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [21]:
agent_chain.run(input="What is ChatGPT?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Calculator: useful for when you need to answer questions about math
Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"

Human: What is ChatGPT?
AI: ChatGPT is a natural language processing tool that is constantly improving.
Human: Who develop

'ChatGPT is a natural language processing tool that was developed by a person.'

In [22]:
agent_chain.run(input="When was the first time you heard about ChatGPT?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Calculator: useful for when you need to answer questions about math
Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"

Human: What is ChatGPT?
AI: ChatGPT is a natural language processing tool that is constantly improving.
Human: Who develop

'I first heard about ChatGPT recently.'